# ML Demo 
- 此文档是机器学习实验的示例模板
- 介绍完整机器学习所需步骤
- 本示例模板也可以认为是benchmark  
- **以后的notebook，都要在最开始写本notebook做的是什么**

## 步骤
- 引入相关依赖包
- 读取数据
- 数据处理（包括但不限于：数据清洗，特征选择，特征工程等）
- 训练模型
- 预测数据
- 指标评价
- 保存

## 引入相关依赖包

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

## 读取数据
- 路径都统一在开始定义
- 等服务器来了，就统一写服务器路径，这样可以确保其他人复制时，可以一键运行
- 全局变量，常量。全部大写，并定义与此

In [2]:
dataOffline_path = 'ccf_offline_stage1_train.csv'
dataPredict_path = 'ccf_offline_stage1_test_revised.csv'
dataPredict_results_path = 'sample_submission.csv'

In [1]:
# 读入训练数据集
dataOffline = pd.read_csv(dataOffline_path)

# 读取预测数据集
dataPredict = pd.read_csv(dataPredict_path)

## 数据处理
#### 此处以distance为例
1. 从训练数据中选取出Distance特征先关的数据
2. 指定用于训练模型的X_train,y_train数据集
3. 从预测数据中选取出Distance特征先关的数据

In [41]:
# 设置类别值
# 使用优惠券，设置为1，否则为0
dataOffline['result'] = dataOffline[['Coupon_id', 'Date']].apply(lambda x: 1 if pd.notnull(x[0]) and pd.notnull(x[1]) else 0, axis=1)

In [42]:
# 将Distance为空的先关数据去除
dataOffline = dataOffline[pd.notna(dataOffline.Distance)]

In [43]:
# 选取训练数据中Distance特征相关数据
x_train = dataOffline[['Distance']]
y_train = dataOffline.result

In [44]:
# 选取预测数据中Distance先关的数据
x_predict = dataPredict[['Distance']]
# Distance为空的预测数据默认填充为10
x_predict = x_predict.fillna(10)

## 训练模型

In [45]:
# 生成logstic模型
classify = LogisticRegression()
# 用训练数据训练模型
classify.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

## 预测数据

In [46]:
predict_proba_y = classify.predict_proba(x_predict)

## 指标评价
- 这是最重要的环节，指标是衡量之前工作是否有效的唯一标准！！！
- 如果出现了和常识不一样的结果，尝试从以下方面找原因
  - 代码（可以通过“代码检视”解决）
  - 深入研究特征和常识的不一致，也许金子就在这里
- 再强调一遍，我们是数字驱动！！！所以一开始在评价指标的确定上，可以多花点时间！！！

# 输出到制定的文件

In [49]:
dataPredict = dataPredict[['User_id', 'Coupon_id', 'Date_received']]
dataPredict.insert(len(dataPredict.columns), 'probability', predict_proba_y[:,1], True)
dataPredict.to_csv(dataPredict_results_path, header=None, index=None)